In [3]:
import pennylane as qml
import jax
import numpy as numpy
import time

from pennylane import qchem
from jax import numpy as np

jax.config.update("jax_enable_x64", True)

symbols = ["Li", "H"]
geometry = np.array([[0.0, 0.0, 0.0], [0.0, 0.0, 2.969280527]])
molecule = qchem.Molecule(symbols, geometry)

In [11]:
H, qubits = qchem.molecular_hamiltonian(
    symbols, 
    geometry,
    active_electrons=2,
    active_orbitals=5
)

qubits = int(qubits) # force qubits to be a simple int 

active_electrons = 2

singles, doubles = qchem.excitations(active_electrons, qubits)

print(f"Total number of excitations = {len(singles) + len(doubles)}")

Total number of excitations = 24


Adaptive Optimizer
- grows an input quantum circuit by adding and optimizing gates selected from a user-defined collection of operators

In [20]:
# # first create an operator pool which contains all single and double excitations
# singles_excitations = [qml.SingleExcitation(0.0, x) for x in singles]
# doubles_excitations = [qml.DoubleExcitation(0.0, x) for x in doubles]
# operator_pool = doubles_excitations + singles_excitations
# FORCE all wire indices to plain int
from pennylane import SingleExcitation, DoubleExcitation

operator_pool = []

# Add all single excitations
for (i, j) in singles:
    op = SingleExcitation(0.0, wires=[i, j])  # 0.0 as dummy param
    operator_pool.append(op)

# Add all double excitations
for (i, j, k, l) in doubles:
    op = DoubleExcitation(0.0, wires=[i, j, k, l])  # 0.0 as dummy param
    operator_pool.append(op)

new_operator_pool = []
for g in operator_pool:
    new_params = [float(p) for p in g.parameters]
    new_gate = type(g)(*new_params, wires=g.wires)
    new_operator_pool.append(new_gate)

operator_pool = new_operator_pool



In [21]:
# now define initial circuit that prepares a Hartree-Fock state and returns the expectation value of a Hamiltonian. Also define a device.
hf_state = qchem.hf_state(active_electrons, qubits)
dev = qml.device("default.qubit", wires=qubits)
@qml.qnode(dev)
def circuit():
    [qml.PauliX(i) for i in np.nonzero(hf_state)[0]]
    return qml.expval(H)

In [23]:
print("Operator pool types:")
for g in operator_pool:
    print(type(g.parameters[0]))

print("Circuit input types:")
print(circuit.interface)

import pennylane as qml
print(qml.version())

for g in operator_pool:
    print(g.wires)

for g in operator_pool:
    for w in g.wires:
        print(w, type(w))

print(type(qubits))
print(qubits)

# instantiate the optimizer and use it to build the circuit adaptively
opt = qml.optimize.AdaptiveOptimizer()
for i in range(len(operator_pool)):
    circuit, energy, gradient = opt.step_and_cost(circuit, operator_pool)
    if i % 3 == 0:
        print("n = {:},  E = {:.8f} H, Largest Gradient = {:.3f}".format(i, energy, gradient))
        print(qml.draw(circuit, decimals=None)())
        print()
    if gradient < 3e-3:
        break

Operator pool types:
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
<class 'float'>
Circuit input types:
autograd
0.34.0
<Wires = [0, 2]>
<Wires = [0, 4]>
<Wires = [0, 6]>
<Wires = [0, 8]>
<Wires = [1, 3]>
<Wires = [1, 5]>
<Wires = [1, 7]>
<Wires = [1, 9]>
<Wires = [0, 1, 2, 3]>
<Wires = [0, 1, 2, 5]>
<Wires = [0, 1, 2, 7]>
<Wires = [0, 1, 2, 9]>
<Wires = [0, 1, 3, 4]>
<Wires = [0, 1, 3, 6]>
<Wires = [0, 1, 3, 8]>
<Wires = [0, 1, 4, 5]>
<Wires = [0, 1, 4, 7]>
<Wires = [0, 1, 4, 9]>
<Wires = [0, 1, 5, 6]>
<Wires = [0, 1, 5, 8]>
<Wires = [0, 1, 6, 7]>
<Wires = [0, 1, 6, 9]>
<Wires = [0, 1, 7, 8]>
<Wires = [0, 1, 8, 9]>
0 <class 'int'>
2 <class 'int'>
0 <class 'int'>
4 <cla

WireError: Wires must be hashable; got object of type <class 'jaxlib.xla_extension.ArrayImpl'>.